# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,39.29,87,100,11.50,US,2025-04-12
1,1,vinh,18.6667,105.6667,66.97,88,100,5.68,VN,2025-04-12
2,2,stanley,54.8680,-1.6985,57.52,61,0,5.75,GB,2025-04-12
3,3,afaahiti,-17.7500,-149.2833,77.47,77,56,2.26,PF,2025-04-12
4,4,grytviken,-54.2811,-36.5092,45.91,84,63,16.75,GS,2025-04-12


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [26]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color= "City",
    frame_width = 700
)

# Display the map
map1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df=(city_data_df[city_data_df["Max Temp"]<80]
          [city_data_df["Max Temp"]>68]
          [city_data_df["Wind Speed"]<10]
          [city_data_df["Cloudiness"]==0])

# Drop any rows with null values
ideal_df.dropna(inplace=True)

# Display sample data
ideal_df.head()

C:\Users\kithk\AppData\Local\Temp\ipykernel_4208\810334917.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ideal_df=(city_data_df[city_data_df["Max Temp"]<80]
C:\Users\kithk\AppData\Local\Temp\ipykernel_4208\810334917.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ideal_df=(city_data_df[city_data_df["Max Temp"]<80]
C:\Users\kithk\AppData\Local\Temp\ipykernel_4208\810334917.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ideal_df=(city_data_df[city_data_df["Max Temp"]<80]


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,pisco,-13.7000,-76.2167,73.45,68,0,9.22,PE,2025-04-12
33,33,santa ana,33.7456,-117.8678,76.44,58,0,5.75,US,2025-04-12
56,56,port augusta,-32.5000,137.7667,70.18,41,0,3.98,AU,2025-04-13
119,119,galliano,29.4422,-90.2992,71.60,40,0,5.75,US,2025-04-12
134,134,adolfo lopez mateos,28.4667,-107.3000,78.49,5,0,5.08,MX,2025-04-12


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
14,pisco,PE,-13.7000,-76.2167,68,
33,santa ana,US,33.7456,-117.8678,58,
56,port augusta,AU,-32.5000,137.7667,41,
119,galliano,US,29.4422,-90.2992,40,
134,adolfo lopez mateos,MX,28.4667,-107.3000,5,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [21]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    hlat=hotel_df.loc[index, "Lat"]
    hlng=hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{hlng},{hlat},{radius}"
    params["bias"] = f"proximity:{hlng},{hlat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    cod=name_address.status_code
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pisco - nearest hotel: La Portada
santa ana - nearest hotel: Holiday Inn Express
port augusta - nearest hotel: Hotel Commonwealth
galliano - nearest hotel: Fairfield Inn & Suites Cut Off-Galliano
adolfo lopez mateos - nearest hotel: No hotel found
tahta - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
inhambane - nearest hotel: Hotel de Inhambane
callao - nearest hotel: Casa Ronald
laguna - nearest hotel: Holiday Inn Express & Suites


,City,Country,Lat,Lng,Humidity,Hotel Name
14,pisco,PE,-13.7000,-76.2167,68,La Portada
33,santa ana,US,33.7456,-117.8678,58,Holiday Inn Express
56,port augusta,AU,-32.5000,137.7667,41,Hotel Commonwealth
119,galliano,US,29.4422,-90.2992,40,Fairfield Inn & Suites Cut Off-Galliano
134,adolfo lopez mateos,MX,28.4667,-107.3000,5,No hotel found
268,tahta,EG,26.7693,31.5021,31,No hotel found
285,mount isa,AU,-20.7333,139.5000,60,Ibis Styles
346,inhambane,MZ,-23.8650,35.3833,83,Hotel de Inhambane
399,callao,PE,-12.0667,-77.1500,64,Casa Ronald
509,laguna,US,38.4210,-121.4238,57,Holiday Inn Express & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 300,
    color = "City",
    hover_cols=["Hotel Name","Country"]
)

# Display the map
map2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)